# MODELADO

## NOMBRE MODELO


<hr>

<code> **Proyecto de Datos II** </code>

## Índice

- [Importación de los datos](#importación-de-los-datos)
- [Preprocesamiento](#preprocesamiento)
- [Entrenamiento](#entrenamiento)
- [Análisis del modelo](#análisis-del-modelo)
- [Registro del modelo en MLflow](#registro-del-modelo-en-mlflow)


In [ ]:
import time
import mlflow
import pandas as pd
from evaluation.evaluator import Evaluator

SEED = 22 # replicabilidad

# =====================================
MODEL_NAME = "" # Rellenar
# =====================================

## Importación de los datos

In [ ]:
df_train = pd.read_parquet("../data/train.parquet")
df_test = pd.read_parquet("../data/test.parquet")

# ! NOTA -> están el ICAO, Callsign y Timestamp por si hay que depurar
X_train, y_train = df_train.drop(columns="takeoff_time", axis=1), df_train["takeoff_time"]
X_test, y_test = df_test.drop(columns="takeoff_time", axis=1), df_test["takeoff_time"]

In [ ]:
X_train.shape, X_test.shape

## Preprocesamiento

In [ ]:
# =====================================
# Rellenar [explicar decisiones]
# =====================================

## Entrenamiento

In [ ]:
start_time = time.time()

# ========================================
# ENTRENAMIENTO AQUÍ
# ========================================

end_time = time.time()
execution_time = end_time - start_time

## Análisis del modelo

In [ ]:
# ===============================================================
mae_train = ...
rmse_train = ...

mae_val = ...
rmse_val = ...
# ===============================================================

In [ ]:
# ===============================================================
# Generar predicciones en test
y_pred = ...
# ===============================================================

df_test['prediction'] = y_pred

In [ ]:
# Nota: df_test tiene que tener la columna 'prediction'
ev = Evaluator(df_test, MODEL_NAME, mae_val, rmse_val)
report = ev.getReport()
ev.visualEvaluation()

### Influencia de las variables

In [ ]:
# ===============================================================
# INFLUENCIA DE LAS VARIABLES
# Rellenar solo si es posible (regresión lineal, random forest)
# ===============================================================

## Registro del modelo en MLflow

In [ ]:
mlflow.set_experiment("takeoff_time_prediction")
mlflow.set_tracking_uri("file:./mlflow_experiments")

with mlflow.start_run():

    # - Datos generales -

    # ========================================================================
    mlflow.set_tag("model_type", MODEL_NAME)
    mlflow.set_tag("framework", "") # scikit-learn, tensorflow, etc.
    mlflow.set_tag("target_variable", "takeoff_time") # variable respuesta
    mlflow.set_tag("preprocessing", "") # transformaciones separadas por un +
    mlflow.set_tag("dataset", "original") # indicar si se ha modificado el conjunto de datos
    mlflow.set_tag("seed", SEED) # semilla para replicabilidad
    # ========================================================================
    
    # - Hiperparámetros óptimos -
    
    # =====================================
    # AÑADIR HIPERPARÁMETROS
    mlflow.log_param("model", MODEL_NAME)
    # =====================================
    
    # - Métricas -

    mlflow.log_metric("execution_time_s", execution_time)

    mlflow.log_metric("mae_val", mae_val)
    mlflow.log_metric("rmse_val", rmse_val)

    mlflow.log_metric("mae_train", mae_train)
    mlflow.log_metric("rmse_train", rmse_train)

    # Registrar métricas globales en test
    for metric_name, value in report["global"].items():
        mlflow.log_metric(f"{metric_name}_test", value)
    
    # Registrar métricas por runway
    for runway, metrics in report["by_runway"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_runway_{runway}", value)
    
    # Registrar métricas por holding point
    for hp, metrics in report["by_holding_point"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_hp_{hp}", value)

    # - Modelo -

    # ========================================================================
    # NOTA - Dependiendo de con qué has hecho el modelo esto hay que cambiarlo
    mlflow.CAMBIAR.log_model(model, MODEL_NAME)
    # ========================================================================
    